In [ ]:
# @title Show job status
!pip install paramiko

import ipywidgets as widgets
from IPython.display import HTML, display, Javascript
import getpass
import paramiko
import stat
import os
import time as timer
import subprocess
from urllib.parse import parse_qs
import sys
from urllib.parse import urlparse, parse_qs
import requests
from google.colab import output

# # JavaScript code to get the current URL
# js_code = '''
# var currentUrl = window.location.href;
# google.colab.kernel.invokeFunction('printUrl', [currentUrl], {});
# '''

# # Define a Python function to print the URL
# def print_url(url):
#     print("Current URL:", url)

# # Register the Python function with Google Colab
# output.register_callback('printUrl', print_url)

# # Execute the JavaScript code
# output.eval_js(js_code)


cluster_input = widgets.Text(value="hostname", description="Cluster:")
username_input = widgets.Text(value="username", description="User name:")
user_password = widgets.Password(placeholder='Enter password',
    description='Password:',
    disabled=False
)
job_id_input = widgets.Text(value="job_id", description="Enter job id:")
submit_button = widgets.Button(description="Submit Job")

input_accordion = widgets.Accordion(children=[widgets.VBox([cluster_input, username_input, user_password, job_id_input, submit_button])])
input_accordion.set_title(0, 'Enter details to see job status/output')
display(input_accordion)


# Function to be executed on button click
def on_submit_button_click(button):
  hostname = cluster_input.value
  username = username_input.value
  password = user_password.value
  job_id = job_id_input.value
  ssh_client = paramiko.SSHClient()
  ssh_client.load_system_host_keys()
  ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
  try:
      ssh_client.connect(hostname, username=username, password=password, allow_agent=True)
  except:
      pass
  transport = ssh_client.get_transport()
  #transport.auth_password(username, getpass.getpass('Enter {0} Logon password :'.format(hostname)))
  sftp_client = paramiko.SFTPClient.from_transport(transport)


  # job status
  # Check the job status periodically
  while True:
    _, stdo, stde = ssh_client.exec_command(f"squeue -u {username} -j {job_id}")
    job_status = stdo.read().decode()
    print(job_status)

    # Break the loop if the job is completed or failed
    if job_id not in job_status:
        break

    # Wait for a few seconds before checking again
    timer.sleep(10)

  print("Output")
  # View output
  _, stdo, stde = ssh_client.exec_command(f"cat /home/{username}/*.o{job_id}")
  print(stdo.read().decode())

  print("Error")
  # View error
  _, stdo, stde = ssh_client.exec_command(f"cat /home/{username}/*.e{job_id}")
  print(stdo.read().decode())


  sftp_client.close()
  ssh_client.close()
  transport.close()

# Attach the function to the button click event
submit_button.on_click(on_submit_button_click)